In [1]:
# Initiate DB
from src.db_models.base_model import db
from src.db_models.sample import Sample
from src.db_models.ml_model import MlModel
from src.db_models.prediction import Prediction
from src.db_models.test_sample import TestSample

# Connect to DB
db.connect()
# Migrate tables
db.create_tables([MlModel, TestSample, Sample, Prediction])


In [2]:
# Get data
import pandas as pd

data = pd.read_csv("../models/Boiler_emulator_dataset.csv")

predictions = data["Class"]
features = data.drop(columns=['Condition', 'Class'])

In [3]:
# Create Primary model
from src.utils.bucket_utils import upload_file_to_bucket, DATASET_FILE_NAME

upload_file_to_bucket(DATASET_FILE_NAME)
MlModel.update(is_active=False).execute()
for model_file, is_active in [("keras_model_1.keras", False), ("keras_model_2.keras", False), ("keras_model_best.keras", True)]:
    upload_file_to_bucket(model_file)
    model_record = MlModel.create(model_path=model_file, is_active=is_active)

2025-04-17 18:02:06.689837: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 18:02:06.696922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744905726.705536   50320 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744905726.708044   50320 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 18:02:06.716878: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
# Fill DB with data

from sklearn.model_selection import train_test_split

X_train_full , X_test , y_train_full , y_test = train_test_split(features, predictions, random_state=42)

# Insert to DB
for index, row in X_test.iterrows():
    sample = TestSample.create(
        Fuel_Mdot = row["Fuel_Mdot"],
        Tair = row["Tair"],
        Treturn = row["Treturn"],
        Tsupply = row["Tsupply"],
        Water_Mdot = row["Water_Mdot"],
        prediction = y_test[index])
        
X_test

,Fuel_Mdot,Tair,Treturn,Tsupply,Water_Mdot
8834,1,283,333.0,343.920469,4.5
626,1,289,333.0,351.416977,4.0
20841,2,293,333.0,345.947128,5.0
5549,3,293,333.0,339.742323,10.0
13218,3,283,333.0,342.588395,3.5
...,...,...,...,...,...
18387,1,289,333.0,335.032964,12.0
20472,2,289,333.0,340.805189,6.5
410,1,287,333.0,359.279102,3.5
21362,2,297,333.0,337.628698,11.0


In [5]:
db.close()

True